In [1]:
# Copyright 2018 Anar Amirli
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train/Test Set Construction
In this document, we construct training and test sets from already computed feature sets. The sets are computed according to the layout:

data<br>
├── train_features<br>
├── test_features<br>

In [2]:
from os import listdir
from os.path import join
import pickle
import numpy as np
import pandas as pd
import multiprocessing
from collections import Counter
from random import shuffle
import json
import re

# our common utility functions that are used in different notebooks
from utils import *

pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)

In [3]:
def describe_dataset(dataset):
    print('Shape: {}\n'.format(dataset.shape))
    
    _, col = dataset.shape
        
    heading = 'Pitch index counts'
    print(heading + '\n' + '-'*len(heading))
    for key, val in sorted(Counter(dataset.values[:, col-1]).items()):
        print('{}\t: {}'.format(int(key), val))

## Pitch IDS

In [4]:
if __name__ == '__main__':

    match_id = 60565
    try:
        features_df = pd.read_csv('../data/general/feature-set/match{}_features_2.csv'.format(match_id))
    except FileNotFoundError:
        pass

    ball_idxs = np.sort(features_df['pitch_index'].unique())
    print('Ball  index: {}'.format(ball_idxs))

    print('Size: {}'.format(len(ball_idxs)))

Ball  index: [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.]
Size: 16


Null index(0) represents all the event categories, apart from the ones we are interested in, that occur when the game stops.

## Dataset Construction

In this section, we construct a combined shuffled dataset for train and set sets, coming from feature data.


### Parameters

When obtaining feature data for a given index, set we should reduce the number of frames of the null ball events in our data set. But cosidering the fact that they are most common, we should keep their numbers close to the number of other most occurred ball location indexes.

* $-$: No-labaled positions intervals would lead to greater label noise.

### Construction
Here we construct the combined dataset from all match data we have in the given data directories.

In [5]:
    # file based constants
    features_dir     = '../data/general/feature-set/'
    features_regex   = re.compile(r'match\d+_features_2.csv')

We speed up the computation by using all the CPU cores via multiprocessing module.

In [6]:
    pool = multiprocessing.Pool()
    df = pd.DataFrame()

    
    features_csv_files = [f for f in listdir(features_dir) if features_regex.match(f)]
    shuffle(features_csv_files)
    print('All matches: {}'.format(len(features_csv_files)))
    features_train_csv = features_csv_files[0:259]
    features_test_csv  = features_csv_files[259:len(features_csv_files)]
    print('Train matches: {}'.format(len(features_train_csv)))
    print('Test matches: {}'.format(len(features_test_csv)))
    
    
    df_train = pd.concat(pool.map(construct_train_set, features_train_csv))
    df_test = pd.concat(pool.map(construct_test_set, features_test_csv))


    print('Train-set shape: {}'.format(df_train.shape))
    print('Test-set shape: {}'.format(df_test.shape))

All matches: 298
Train matches: 259
Test matches: 39
Train-set shape: (229935, 228)
Test-set shape: (99329, 228)


In [13]:
    # shuffling data-sets agin
    df_train = df_train.drop('homeGoalKeeperY', axis=1)
    
    df_train = df_train.sample(frac=1).reset_index(drop=True)
    df_test = df_test.sample(frac=1).reset_index(drop=True)

### Train Set

In [14]:
    describe_dataset(df_train)
    display(df_train.head())

Shape: (229935, 227)

Pitch index counts
------------------
0	: 18404
1	: 4759
2	: 23711
3	: 4955
4	: 14031
5	: 17747
6	: 14061
7	: 20830
8	: 21339
9	: 21304
10	: 16105
11	: 12249
12	: 17671
13	: 7752
14	: 5936
15	: 9081


,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
0,70.00000,24.812500,1.150391,-1.0,1.0,72.6875,24.750000,71.37500,3.990234,77.6250,...,32.843750,2.359375,0.000,0.000,52.21875,32.843750,0.0,0.0,0.0,12.0
1,54.71875,16.234375,2.480469,-1.0,1.0,55.7500,19.468750,50.71875,18.109375,66.9375,...,15.000000,1.929688,0.000,0.000,41.28125,15.000000,0.0,0.0,0.0,7.0
2,68.56250,19.343750,2.390625,1.0,-1.0,67.8125,12.148438,57.71875,26.078125,75.0625,...,17.234375,2.660156,0.000,0.000,62.50000,17.234375,0.0,0.0,0.0,7.0
3,70.31250,37.562500,1.540039,1.0,-1.0,54.5625,45.156250,51.65625,44.093750,81.1250,...,52.375000,1.594727,0.000,0.000,51.59375,52.375000,0.0,0.0,0.0,9.0
4,45.84375,44.750000,1.280273,1.0,1.0,35.6250,61.375000,32.71875,50.437500,68.3125,...,61.250000,1.785156,35.625,61.375,32.37500,61.125000,0.0,0.0,0.0,9.0


### Test Set

In [15]:
    describe_dataset(df_test)
    display(df_test.head())

Shape: (99329, 228)

Pitch index counts
------------------
0	: 65259
1	: 850
2	: 4080
3	: 766
4	: 2308
5	: 2912
6	: 2407
7	: 3327
8	: 3331
9	: 3477
10	: 2469
11	: 1959
12	: 2683
13	: 1108
14	: 935
15	: 1458


,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
0,79.125000,29.718750,1.349609,-1.0,-1.0,82.93750,38.656250,79.875000,39.312500,88.5000,...,45.031250,1.900391,0.000000,0.000000,80.937500,45.03125,0.0000,0.00000,0.000000,12.0
1,68.625000,34.718750,4.539062,-1.0,-1.0,86.18750,43.500000,67.000000,34.218750,95.0625,...,29.406250,4.160156,0.000000,0.000000,0.000000,0.00000,46.0625,29.40625,4.160156,0.0
2,23.812500,35.718750,2.449219,1.0,-1.0,18.37500,31.765625,18.937500,30.031250,28.2500,...,42.625000,1.615234,12.828125,45.156250,11.320312,40.06250,0.0000,0.00000,0.000000,2.0
3,27.296875,29.078125,1.700195,1.0,1.0,30.12500,26.265625,31.390625,30.359375,51.9375,...,30.453125,1.389648,23.671875,30.453125,0.000000,0.00000,0.0000,0.00000,0.000000,0.0
4,60.625000,22.593750,0.290039,-1.0,-1.0,55.53125,23.515625,51.093750,27.765625,69.1250,...,24.921875,1.495117,43.718750,30.656250,45.062500,19.18750,0.0000,0.00000,0.000000,4.0


## Exporting

In [16]:
    df_train.to_csv('../data/general/{dataset}/all_{dataset}_2.csv'.format(dataset='train'), index=False)
    df_test.to_csv('../data/general/{dataset}/all_{dataset}_2.csv'.format(dataset='test'), index=False)

In [17]:
    df_train[1:10]

,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
1,54.71875,16.234375,2.480469,-1.0,1.0,55.75000,19.468750,50.71875,18.109375,66.9375,...,15.000000,1.929688,0.00000,0.000000,41.28125,15.000000,0.000,0.00000,0.000000,7.0
2,68.56250,19.343750,2.390625,1.0,-1.0,67.81250,12.148438,57.71875,26.078125,75.0625,...,17.234375,2.660156,0.00000,0.000000,62.50000,17.234375,0.000,0.00000,0.000000,7.0
3,70.31250,37.562500,1.540039,1.0,-1.0,54.56250,45.156250,51.65625,44.093750,81.1250,...,52.375000,1.594727,0.00000,0.000000,51.59375,52.375000,0.000,0.00000,0.000000,9.0
4,45.84375,44.750000,1.280273,1.0,1.0,35.62500,61.375000,32.71875,50.437500,68.3125,...,61.250000,1.785156,35.62500,61.375000,32.37500,61.125000,0.000,0.00000,0.000000,9.0
5,73.06250,34.437500,2.419922,1.0,1.0,74.75000,47.093750,67.93750,42.468750,87.5625,...,43.125000,1.150391,53.06250,43.125000,0.00000,0.000000,0.000,0.00000,0.000000,12.0
6,74.43750,25.343750,1.099609,-1.0,-1.0,72.25000,33.718750,72.12500,31.687500,79.2500,...,40.343750,3.199219,0.00000,0.000000,0.00000,0.000000,61.750,40.34375,3.199219,7.0
7,50.40625,37.437500,0.939941,-1.0,-1.0,55.96875,33.468750,48.68750,33.406250,71.7500,...,29.421875,1.150391,44.65625,29.421875,0.00000,0.000000,0.000,0.00000,0.000000,0.0
8,53.59375,31.453125,1.719727,-1.0,-1.0,54.40625,43.156250,50.75000,37.906250,67.5000,...,34.062500,1.610352,39.28125,28.703125,48.96875,39.406250,0.000,0.00000,0.000000,5.0
9,68.87500,21.796875,0.799805,1.0,-1.0,74.43750,32.531250,72.12500,29.578125,80.6875,...,25.343750,3.009766,0.00000,0.000000,0.00000,0.000000,62.625,25.34375,3.009766,8.0
